In [1]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Raster_Centroid, Raster_BBox, Edges, Line_Intersection, Raster_Scanline, Scanline_Rasterize_Polygon

In [2]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

In [3]:
image_resolution = 1080
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)


In [4]:
imageArray = np.array(img)
print(imageArray)
def Weighted_Raster_Centroid(pixels):
    cx = 0
    cy = 0
    total_weight = 0
    for pixel in pixels:
        sample_x = pixel[1]
        sample_y = pixel[0]
        # sampled_value = image.getpixel((sample_x,sample_y))
        sampled_value = imageArray[int(sample_x)][int(sample_y)]

        weight = 1 - np.mean(sampled_value) / 255

        if weight < 0.05:
            weight = 0

        total_weight += weight
        
        cx += pixel[0] * weight
        cy += pixel[1] * weight
    
    if total_weight == 0:
        return False

    cx /= total_weight
    cy /= total_weight
    
    return [cx, cy]

In [5]:
def Clamp(n, range):
    return max(range[0], min(range[1], n))

In [6]:
polygon = np.array([[150,-100],[750,200],[1200,400],[700,800],[500,900],[-75,500]])

for point in polygon:
    center_ellipse(point[0],point[1],5,'red')
bbox = Raster_BBox(polygon, image_resolution)
rasterized_polygon = Scanline_Rasterize_Polygon(polygon,bbox,1080)
x_bounds = [0,1080]
y_bounds = [0,1080]
for point in rasterized_polygon:
    if point[0] < x_bounds[0]:
        x_bounds[0] = point[0]
    if point[0] > x_bounds[1]:
        x_bounds[1] = point[0]
    if point[1] < y_bounds[0]:
        y_bounds[0] = point[1]
    if point[1] > y_bounds[1]:
        y_bounds[1] = point[1]
print(x_bounds)
print(y_bounds)
# print(rasterized_polygon)

[0, 1080]
[0, 1080]


In [7]:
centroidW = Weighted_Raster_Centroid(rasterized_polygon)
centroid = Raster_Centroid(rasterized_polygon)
for pixel in rasterized_polygon:
    center_rectangle(pixel[0],pixel[1],1,1,'gray')

center_ellipse(centroid[0],centroid[1],5,'green')
center_ellipse(centroidW[0],centroidW[1],10,'white')
img.save('centroid_test.png')